# Load checkpoint and imports

In [1]:
import os
import sys
parent_dir = "/Midgard/home/farzantn/phd/Olfaction/MoLFormer_N2024"
sys.path.append(parent_dir)
parent_dir="/Midgard/home/farzantn/mambaforge/envs/MolTran_CUDA11_cuda/lib/python3.8"
sys.path.append(parent_dir)

In [2]:
device_name='cuda'
# 
# from datetime import datetime
# from argparse import Namespace
# import yaml
import os
# import pandas as pd
# from rdkit import Chem
# from sklearn.linear_model import LogisticRegression
print(os.getcwd())
# import torch
# from sklearn.decomposition import PCA
# import matplotlib.pyplot as plt
# import numpy as np
from custom_utils.tokenizer.tokenizer import MolTranBertTokenizer
# from sklearn.model_selection import train_test_split
# import rdkit
# 
# from openpom.utils.data_utils import get_class_imbalance_ratio
# from openpom.feat.graph_featurizer import GraphFeaturizer, GraphConvConstants
# from openpom.utils.loss import CustomMultiLabelLoss
print(os.getcwd())
# from utils.mol_loss import MolCustomMultiLabelLoss
# from openpom.utils.loss import CustomMultiLabelLoss

# from openpom.feat.graph_featurizer import GraphFeaturizer, GraphConvConstants

# import deepchem as dc
# import rdkit
from utils.gs_lf import *
with open('../custom_utils/hparams.yaml', 'r') as f:
    config = Namespace(**yaml.safe_load(f))
print(torch.cuda.is_available())
# from fast_transformers.masking import LengthMask as LM
from custom_utils.train_pubchem_light import LightningModule
# from openpom.utils.optimizer import get_optimizer
# from torch import nn
# import torch.nn.functional as F
torch.cuda.is_available()
torch.cuda.current_device()
# from utils.gs_lf import *
# import seaborn as sns
# import pyrfume
# from utils.util_alignment import *
# from utils.prepare_datasets import *
# from utils.custom_models import *
# from sklearn.preprocessing import StandardScaler
# from scipy.stats import pearsonr  
# from sklearn.metrics import r2_score
# import itertools
# import math
from utils.util_alignment import *
# from utils.custom_models import *
# import seaborn as sns
# from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
# from matplotlib.patches import Patch
# from scipy.stats import gaussian_kde
# import numpy as np
# import os
# from IPython.display import Image
# from openpom.models.mpnn_pom import MPNNPOMModel
# plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 25
plt.rcParams["figure.figsize"] = (10,10)
# plt.rc('font',**{'family':'serif','serif':['Times']})
# from ast import literal_eval
# from utils.ridge_tools import *
# import ast
# from scipy import stats
# from sklearn.metrics import roc_auc_score, mean_squared_error

|# Loading Models

## Loading MolFormer Model

In [3]:
tokenizer = MolTranBertTokenizer('../custom_utils/tokenizer/bert_vocab.txt')
ckpt = '../MoLformer_Pretrained/checkpoints/N-Step-Checkpoint_3_30000.ckpt'
lm = LightningModule(config, tokenizer.vocab).load_from_checkpoint(ckpt, config=config, vocab=tokenizer.vocab)

# Extracting embedding from MoLFormer

## helper methods

In [4]:
def convert_todf_molformer(embeddings_dataset,cids,subjects=None,y=None):
    embeddings_dataset = pd.DataFrame(embeddings_dataset)
    embeddings_dataset['embeddings'] = embeddings_dataset.loc[:, 0:768].values.tolist()
    embeddings_dataset['CID'] = cids
    if subjects is not None:
        embeddings_dataset['subject'] = subjects
        
    if y is not None:
        y_dataset = pd.DataFrame(y)
        y_dataset['y'] = y_dataset.loc[:, 0:768].values.tolist()
    
        df = pd.concat([embeddings_dataset, y_dataset], axis=1)
        return df
    else:
        return embeddings_dataset

### Sagar

In [ ]:
input_file_sagar = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/mols_datasets/curated_sagar_subjects_nonaminus.csv' # or new downloaded file path
# '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/embeddings/molformer/gslf_molformer_embeddings_13_Apr17.csv'
smiles_field = 'nonStereoSMILES'
df_sagar_temp=pd.read_csv(input_file_sagar)
sagar_tasks= df_sagar_temp.columns.to_list()[1:16]
cids_sagar= df_sagar_temp['cid'].values.tolist()
subjects_sagar= df_sagar_temp['subject'].values.tolist()


In [ ]:
X_sagar,y_sagar,X_layers_sagar,y_layers_sagar=extract_molformer_representations(lm, tokenizer, sagar_tasks, input_file_sagar, smiles_field)
df_embeddings_sagar = convert_todf_molformer(X_sagar,cids_sagar,subjects_sagar,y_sagar)
df_embeddings_sagar.to_csv('sagar_molformer_embeddings_13_Apr17.csv', index=False)
for i,X in enumerate(X_layers_sagar):
    print("i",i)
    df_embeddings_sagar = convert_todf_molformer(X_layers_sagar[i],cids_sagar,subjects_sagar,y_layers_sagar[i])
    df_embeddings_sagar.to_csv('sagar_molformer_embeddings_'+str(i)+'_Apr17.csv', index=False)

In [ ]:
# df_embeddings_sagar.columns.values

### Keller

In [ ]:
input_file_keller = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/mols_datasets/curated_keller2016_nona.csv' # or new downloaded file path
smiles_field = 'nonStereoSMILES'
df_keller_temp=pd.read_csv(input_file_keller)
keller_tasks= df_keller_temp.columns.to_list()[5:]
cids_keller= df_keller_temp['CID'].values.tolist()
cids_subject= df_keller_temp['Subject'].values.tolist()
X_keller,y_keller,X_layers_keller,y_layers_keller=extract_molformer_representations(lm, tokenizer, keller_tasks, input_file_keller, smiles_field)

In [ ]:
df_embeddings_keller = convert_todf_molformer(X_keller,cids_keller,cids_subject,y_keller)
df_embeddings_keller.to_csv('keller_molformer_embeddings_13_Apr17.csv', index=False)
for i,X in enumerate(X_layers_keller):
    print("i",i)
    df_embeddings_keller = convert_todf_molformer(X_layers_keller[i],cids_keller,cids_subject,y_layers_keller[i])
    df_embeddings_keller.to_csv('keller_molformer_embeddings_'+str(i)+'_Apr17.csv', index=False)

In [ ]:
# df_embeddings_keller.columns.values.tolist()

### Ravia

In [ ]:
input_file_ravia = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/mols_datasets/ravia_molecules.csv'
df_ravia_temp=pd.read_csv(input_file_ravia)
smiles_field = 'nonStereoSMILES'
cids_ravia= df_ravia_temp['CID'].values.tolist()
X_ravia,y_ravia,X_layers_ravia,y_layers_ravia=extract_molformer_representations(lm, tokenizer, [], input_file_ravia, smiles_field)

In [ ]:
df_embeddings_ravia = convert_todf_molformer(X_ravia,cids_ravia,None,y_ravia)
df_embeddings_ravia.to_csv('ravia_molformer_embeddings_13_Apr17.csv', index=False)
for i,X in enumerate(X_layers_ravia):
    # print("i",i)
    df_embeddings_ravia = convert_todf_molformer(X_layers_ravia[i],cids_ravia,None,y_layers_ravia[i])
    df_embeddings_ravia.to_csv('ravia_molformer_embeddings_'+str(i)+'_Apr17.csv', index=False)

In [ ]:
# df_embeddings_ravia

### Snitz

In [ ]:
input_file_snitz = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/mols_datasets/snitz_molecules.csv'
df_snitz_temp=pd.read_csv(input_file_snitz)
smiles_field = 'nonStereoSMILES'
cids_snitz= df_snitz_temp['CID'].values.tolist()
X_snitz,y_snitz,X_layers_snitz,y_layers_snitz=extract_molformer_representations(lm, tokenizer, [], input_file_snitz, smiles_field)

In [ ]:
df_embeddings_snitz = convert_todf_molformer(X_snitz,cids_snitz,None,y_snitz)
df_embeddings_snitz.to_csv('snitz_molformer_embeddings_13_Apr17.csv', index=False)
for i,X in enumerate(X_layers_snitz):
    print("i",i)
    df_embeddings_snitz = convert_todf_molformer(X_layers_snitz[i],cids_snitz,y_layers_snitz[i])
    df_embeddings_snitz.to_csv('snitz_molformer_embeddings_'+str(i)+'_Apr17.csv', index=False)

### GS-LF

In [ ]:
input_file_gslf = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/mols_datasets/curated_GS_LF_merged_4983.csv' # or new downloaded file path
smiles_field = 'nonStereoSMILES'
df_gslf_temp=pd.read_csv(input_file_gslf)
gslf_tasks=df_gslf_temp.columns.to_list()[2:]
cids_gslf= df_gslf_temp.index.values.tolist()
X_gslf,y_gslf,X_layers_gslf,y_layers_gslf=extract_molformer_representations(lm, tokenizer, gslf_tasks, input_file_gslf, smiles_field)

In [ ]:
# df_gslf_temp.index.values.tolist()

In [ ]:
df_embeddings_gslf = convert_todf_molformer(X_gslf,cids_gslf,None,y_gslf)
df_embeddings_gslf.to_csv('gslf_molformer_embeddings_13_Apr17.csv', index=False)
for i,X in enumerate(X_layers_gslf):
    # print("i",i)
    df_embeddings_gslf = convert_todf_molformer(X_layers_gslf[i],cids_gslf,y_layers_gslf[i])
    df_embeddings_gslf.to_csv('gslf_molformer_embeddings_'+str(i)+'_Apr17.csv', index=False)

## dravienks

In [11]:
input_file_dravienks_1 = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/tasks/dravnieks1985_applicability_1.csv' # or new  
# '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/embeddings/molformer/gslf_molformer_embeddings_13_Apr17.csv'
smiles_field = 'nonStereoSMILES'
df_dravienks_temp_1=pd.read_csv(input_file_dravienks_1)
dravienks_tasks_1= df_dravienks_temp_1.columns.to_list()[1:-9]
cids_dravienks= df_dravienks_temp_1['CID'].values.tolist()
X_dravienks,y_dravienks,X_layers_dravienks,y_layers_dravienks=extract_molformer_representations(lm, tokenizer, dravienks_tasks_1, input_file_dravienks_1, smiles_field)

In [12]:
df_embeddings_dravienks = convert_todf_molformer(X_dravienks,cids_dravienks,None,y_dravienks)
df_embeddings_dravienks.to_csv('dravienks1985App1_molformer_embeddings_13_Apr17.csv', index=False)
for i,X in enumerate(X_layers_dravienks):
    print("i",i)
    df_embeddings_dravienks = convert_todf_molformer(X_layers_dravienks[i],cids_dravienks,None,y_layers_dravienks[i])
    df_embeddings_dravienks.to_csv('dravienks1985App1_molformer_embeddings_'+str(i)+'_Apr17.csv', index=False)


In [13]:
input_file_dravienks_2 = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/tasks/dravnieks1985_use_2.csv' # or new downloaded file path
# '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/embeddings/molformer/gslf_molformer_embeddings_13_Apr17.csv'
smiles_field = 'nonStereoSMILES'
df_dravienks_temp_2=pd.read_csv(input_file_dravienks_2)
dravienks_tasks_2= df_dravienks_temp_2.columns.to_list()[1:-9]
cids_dravienks= df_dravienks_temp_2['CID'].values.tolist()
X_dravienks,y_dravienks,X_layers_dravienks,y_layers_dravienks=extract_molformer_representations(lm, tokenizer, dravienks_tasks_2, input_file_dravienks_2, smiles_field)

In [14]:
df_embeddings_dravienks = convert_todf_molformer(X_dravienks,cids_dravienks,None,y_dravienks)
df_embeddings_dravienks.to_csv('dravienks1985Use2_molformer_embeddings_13_Apr17.csv', index=False)
for i,X in enumerate(X_layers_dravienks):
    print("i",i)
    df_embeddings_dravienks = convert_todf_molformer(X_layers_dravienks[i],cids_dravienks,None,y_layers_dravienks[i])
    df_embeddings_dravienks.to_csv('dravienks1985Use2_molformer_embeddings_'+str(i)+'_Apr17.csv', index=False)